In [229]:
import pandas as pd
from typing import Dict
import h3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import random
import uuid
import json
pd.set_option('display.max_columns', None)

In [230]:
routes = pd.read_csv("../data/gtfs_static/routes.txt")
shapes = pd.read_csv("../data/gtfs_static/shapes.txt")
stops = pd.read_csv("../data/gtfs_static/stops.txt")
trips = pd.read_csv("../data/gtfs_static/trips.txt")

/var/folders/8v/7lq6zc114v7c1x27n8zl0bs40000gn/T/ipykernel_11390/200900448.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("../data/gtfs_static/trips.txt")


In [232]:
routes[routes.route_id == 293191]

,route_long_name,route_type,route_text_color,route_color,agency_id,route_id,route_url,route_desc,route_short_name


In [181]:
stop_times = pd.read_csv("../data/gtfs_static/stop_times.txt")
stop_times[(stop_times.trip_id ==14124532) ]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
1919207,14124532,10:40:00,10:40:00,12362,1,NaN,NaN,NaN,NaN,1
1919208,14124532,10:41:07,10:41:07,1491,2,NaN,NaN,NaN,0.5411,0
1919209,14124532,10:41:55,10:41:55,11499,3,NaN,NaN,NaN,0.9318,0
1919210,14124532,10:42:49,10:42:49,1493,4,NaN,NaN,NaN,1.3732,0
1919211,14124532,10:43:43,10:43:43,11759,5,NaN,NaN,NaN,1.8113,0
...,...,...,...,...,...,...,...,...,...,...
1919264,14124532,11:33:04,11:33:04,2011,58,NaN,NaN,NaN,19.7705,0
1919265,14124532,11:34:34,11:34:34,10331,59,NaN,NaN,NaN,20.1865,0
1919266,14124532,11:36:19,11:36:19,2015,60,NaN,NaN,NaN,20.6711,0
1919267,14124532,11:37:17,11:37:17,1172,61,NaN,NaN,NaN,20.9389,0


In [182]:
def get_h3_coords(lat, lng, resolution):
    lat = float(lat)
    lng = float(lng)
    return h3.geo_to_h3(lat, lng, resolution)


UBC_EXCHANGE_COORDS = (49.267, -123.247)
UBC_EXCHANGE_INDEX = h3.geo_to_h3(UBC_EXCHANGE_COORDS[0], UBC_EXCHANGE_COORDS[1], 7)

In [183]:
starting_shapes = shapes[shapes.shape_pt_sequence == 1]
starting_shapes["h3_7"] = starting_shapes.apply(
    lambda x: get_h3_coords(x.shape_pt_lat, x.shape_pt_lon, 7), axis=1
)

UBC_STARTING_SHAPE_ID = starting_shapes[
    starting_shapes["h3_7"].isin([UBC_EXCHANGE_INDEX])
]["shape_id"].to_list()

/var/folders/8v/7lq6zc114v7c1x27n8zl0bs40000gn/T/ipykernel_11390/1428455417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_shapes["h3_7"] = starting_shapes.apply(


In [184]:
shapes[shapes["shape_id"].isin(UBC_STARTING_SHAPE_ID)]

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
14322,297021,49.265681,-123.249846,1,0.0000
14323,297021,49.265897,-123.248547,2,0.0975
14324,297021,49.266003,-123.247825,3,0.1514
14325,297021,49.266073,-123.247333,4,0.1880
14326,297021,49.266322,-123.245605,5,0.3168
...,...,...,...,...,...
311591,298129,49.277532,-123.123614,118,10.4196
311592,298129,49.278601,-123.122011,119,10.5862
311593,298129,49.279647,-123.120414,120,10.7506
311594,298129,49.280722,-123.118797,121,10.9184


In [185]:
H3_RES = 10

stops_enriched = stops[['stop_id', 'stop_lat', 'stop_lon','stop_name']]
stops_enriched['h3'] = stops_enriched.apply(lambda x: get_h3_coords(x.stop_lat, x.stop_lon, H3_RES), axis=1)
stops_enriched['h3_9'] = stops_enriched.apply(lambda x: get_h3_coords(x.stop_lat, x.stop_lon, 9), axis=1)

# stops_enriched.head().to_dict('records')

/var/folders/8v/7lq6zc114v7c1x27n8zl0bs40000gn/T/ipykernel_11390/1737994490.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stops_enriched['h3'] = stops_enriched.apply(lambda x: get_h3_coords(x.stop_lat, x.stop_lon, H3_RES), axis=1)
/var/folders/8v/7lq6zc114v7c1x27n8zl0bs40000gn/T/ipykernel_11390/1737994490.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stops_enriched['h3_9'] = stops_enriched.apply(lambda x: get_h3_coords(x.stop_lat, x.stop_lon, 9), axis=1)


In [186]:
stops_enriched = stops_enriched.drop_duplicates(subset=['h3'])

In [187]:
ubc_all_trip_shapes = trips.merge(
    shapes[shapes["shape_id"].isin(UBC_STARTING_SHAPE_ID)], on="shape_id"
).merge(routes, on="route_id")

In [188]:
trips[['trip_id', 'route_id', 'shape_id', 'trip_headsign', 'direction_id']]

,trip_id,route_id,shape_id,trip_headsign,direction_id
0,14305517,30052,298080,Millennium Line To VCC-Clark,1
1,14305516,30052,298080,Millennium Line To VCC-Clark,1
2,14041899,34542,292878,406 Railway Ave to Brighouse Station,0
3,14041898,34542,292878,406 Railway Ave to Brighouse Station,0
4,14305513,30052,298080,Millennium Line To VCC-Clark,1
...,...,...,...,...,...
122638,14268220,6796,297285,105 Uptown,0
122639,14268227,6796,297285,105 Uptown,0
122640,14265931,6641,297255,99 Commercial-Broadway Station,0
122641,14268225,6796,297285,105 Uptown,0


In [189]:
trips[[ 'shape_id', 'trip_headsign', 'direction_id']].drop_duplicates(
    [ 'shape_id', 'trip_headsign', 'direction_id']
).to_csv("../data/static_mod/trip_shapes_mapping.csv", index=False)

In [190]:
all_ubc_departing_buses = list(ubc_all_trip_shapes.trip_headsign.unique())
all_ubc_departing_buses.sort()
all_ubc_departing_buses

['14 Downtown',
 '14 Hastings',
 '25 King Edward/To Brentwood Station',
 '25 King Edward/To Nanaimo Station',
 '33 16th & 33RD Avenue/To 29th Avenue Station',
 '4 Downtown',
 '4 Powell',
 '44 Downtown',
 '44 Dundarave Express',
 '49 Metrotown Station',
 '68 Wesbrook Village',
 '84 VCC-Clark Station',
 '9 Boundary',
 '99 Commercial-Broadway Station',
 '99 To Boundary B-Line',
 'N17 Downtown NightBus',
 'R4 41ST Avenue/To Joyce Station']

In [191]:
# ubc_all_trip_shapes[ubc_all_trip_shapes.shape_id == 292297]

In [192]:
distinct_shapes = shapes.shape_id.unique()
sample_shapes = shapes[shapes.shape_id.isin(UBC_STARTING_SHAPE_ID)]
sample_shapes['h3'] = sample_shapes.apply(lambda x: get_h3_coords(x['shape_pt_lat'], x['shape_pt_lon'], H3_RES), axis=1)

/var/folders/8v/7lq6zc114v7c1x27n8zl0bs40000gn/T/ipykernel_11390/2905110465.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_shapes['h3'] = sample_shapes.apply(lambda x: get_h3_coords(x['shape_pt_lat'], x['shape_pt_lon'], H3_RES), axis=1)


In [193]:
stops_enriched

,stop_id,stop_lat,stop_lon,stop_name,h3,h3_9
0,11542,49.278166,-123.126768,Southbound Howe St @ Davie St,8a28de8d8917fff,8928de8d893ffff
1,11543,49.248581,-122.843968,Northbound Laurentian Cres @ Austin Ave,8a28dec6899ffff,8928dec689bffff
2,11540,49.170215,-123.125653,Westbound Westminster Hwy @ Garden City Rd,8a28de81baaffff,8928de81babffff
3,11541,49.219164,-123.150433,Southbound West Blvd @ W 57 Ave,8a28de8c405ffff,8928de8c407ffff
4,11546,49.452388,-123.236624,Northbound Hwy 99 Onramp @ Kelvin Grove Way,8a28dea925a7fff,8928dea925bffff
...,...,...,...,...,...,...
8837,7466,49.090132,-123.071193,Eastbound Ladner Trunk Rd @ Linden Dr,8a28de833457fff,8928de83347ffff
8838,8480,49.334491,-123.158293,Eastbound Inglewood Ave @ 15 St,8a28de16200ffff,8928de16203ffff
8839,8483,49.133342,-123.087853,Eastbound Steveston Hwy @ Hwy 99,8a28de81e937fff,8928de81e93ffff
8843,8488,49.051695,-122.867263,Northbound 128 St @ 26B Ave,8a28de994347fff,8928de99437ffff


In [194]:
sample_shapes_with_stops = sample_shapes.merge(stops_enriched, left_on='h3', right_on='h3', how = 'left')

In [195]:
sample_shapes_with_stops

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,h3,stop_id,stop_lat,stop_lon,stop_name,h3_9
0,297021,49.265681,-123.249846,1,0.0000,8a28debb251ffff,NaN,NaN,NaN,NaN,NaN
1,297021,49.265897,-123.248547,2,0.0975,8a28debb25affff,11793.0,49.265810,-123.248129,UBC Exchange @ Bay 10,8928debb25bffff
2,297021,49.266003,-123.247825,3,0.1514,8a28debb2587fff,11794.0,49.265923,-123.247364,UBC Exchange @ Bay 11,8928debb25bffff
3,297021,49.266073,-123.247333,4,0.1880,8a28debb2587fff,11794.0,49.265923,-123.247364,UBC Exchange @ Bay 11,8928debb25bffff
4,297021,49.266322,-123.245605,5,0.3168,8a28debb2597fff,12483.0,49.266634,-123.244736,Westbound University Blvd @ Western Parkway,8928de8c997ffff
...,...,...,...,...,...,...,...,...,...,...,...
7535,298129,49.277532,-123.123614,118,10.4196,8a28de8dc65ffff,11429.0,49.277584,-123.125372,Northbound Granville St @ Davie St,8928de8dc67ffff
7536,298129,49.278601,-123.122011,119,10.5862,8a28de8dc6cffff,9366.0,49.278835,-123.121579,Northbound Seymour St @ Nelson St,8928de8dc6fffff
7537,298129,49.279647,-123.120414,120,10.7506,8a28de8dc6dffff,NaN,NaN,NaN,NaN,NaN
7538,298129,49.280722,-123.118797,121,10.9184,8a28de8d8d0ffff,12535.0,49.281020,-123.118240,Northbound Seymour St @ Robson St,8928de8d8d3ffff


In [196]:
# sample_shapes_with_stops_297021 = sample_shapes_with_stops[sample_shapes_with_stops.shape_id == 297021]

In [197]:
def remove_duplicates_stops(df):
    # Group by shape_id
    result_dfs = []
    
    for shape_id, group in df.groupby('shape_id'):
        group = group.reset_index(drop=True)
        
        # Process each group
        for idx in range(len(group)):
            l = idx
            r = idx + 1
            
            while r < len(group):
                if group.loc[l, 'stop_name'] == group.loc[r, 'stop_name']:
                    group.loc[r, 'stop_name'] = None
                    r += 1
                else:
                    l = r
                    r += 1
        
        result_dfs.append(group)
    
    # Combine all processed groups
    return pd.concat(result_dfs, ignore_index=True)


In [198]:
sample_shapes_with_stops = remove_duplicates_stops(sample_shapes_with_stops)

In [199]:
# sample_shapes_with_stops.head().to_dict('records')

In [200]:
def simulate_bus_positions_og(
    shapes_df: pd.DataFrame,
    start_time: datetime,
    end_time: datetime,
    interval_seconds: int = 60,
) -> pd.DataFrame:
    """
    Simulate bus movements for each shape_id between `start_time` and `end_time`.

    Returns a DataFrame with:
    [timestamp, bus_id, route_id, direction_id, latitude, longitude]
    """
    # Sort points
    shapes_df = shapes_df.sort_values(["shape_id", "shape_pt_sequence"])

    # We'll collect all simulated rows in a list of dicts
    all_rows = []

    # Let's define a simple function to chunk the route points
    # over a specified time window
    def generate_positions_for_shape(coords, route_id, direction_id):
        """Generate lat/lon positions from the coords in evenly spaced intervals."""
        # Example: We’ll assume the bus goes from coords[0] to coords[-1]
        # in (end_time - start_time).
        bus_uuid = str(uuid.uuid4())

        total_time = (end_time - start_time).total_seconds()
        num_intervals = int(total_time // interval_seconds)

        # We'll simply iterate linearly from coords[0] to coords[-1]
        # More sophisticated approach might measure actual shape_dist_traveled
        # to spread out the bus times.
        for i in range(num_intervals):
            timestamp = start_time + timedelta(seconds=i * interval_seconds)
            # fraction of route completed
            frac = i / max(num_intervals - 1, 1)
            # index in coords
            idx_float = frac * (len(coords) - 1)
            idx_lower = int(np.floor(idx_float))
            idx_upper = int(np.ceil(idx_float))

            if idx_lower == idx_upper:
                lat, lon = coords[idx_lower]
            else:
                # Linear interpolation between coords[idx_lower] and coords[idx_upper]
                lat_lower, lon_lower = coords[idx_lower]
                lat_upper, lon_upper = coords[idx_upper]
                ratio = idx_float - idx_lower
                lat = lat_lower + (lat_upper - lat_lower) * ratio
                lon = lon_lower + (lon_upper - lon_lower) * ratio

            all_rows.append(
                {
                    "timestamp": timestamp,
                    "bus_id": f"Bus_{route_id}_{direction_id}",  # or generate unique
                    "route_id": route_id,
                    "direction_id": direction_id,
                    "latitude": lat,
                    "longitude": lon,
                    "bus_id": bus_uuid,   
                }
            )

    # Group by shape_id
    for shape_id, group in shapes_df.groupby("shape_id"):
        # Extract lat/lon coords
        coords = list(zip(group["shape_pt_lat"].values, group["shape_pt_lon"].values))

        # Simulate forward direction
        generate_positions_for_shape(coords, shape_id, direction_id=1)

        # Simulate backward direction (just reverse coords)
        coords_reversed = coords[::-1]
        generate_positions_for_shape(coords_reversed, shape_id, direction_id=0)

    # Convert all_rows to DataFrame
    simulated_df = pd.DataFrame(all_rows)
    return simulated_df


# # Example usage:
# shapes_data = pd.DataFrame(
#     {
#         "shape_id": [298135, 298135, 298135, 298135],
#         "shape_pt_lat": [49.285278, 49.284275, 49.283132, 49.282477],
#         "shape_pt_lon": [-123.115845, -123.117371, -123.119153, -123.118136],
#         "shape_pt_sequence": [1, 2, 3, 4],
#         "shape_dist_traveled": [0.0, 0.1574, 0.339, 0.4428],
#     }
# )

In [201]:
manual_wesbrook_agronomy = (49.26412604364934, -123.24368370667061)
manual_wesbrook_agronomy_index = get_h3_coords(manual_wesbrook_agronomy[0], manual_wesbrook_agronomy[1], 9)
manual_wesbrook_agronomy_index

'8928de8cd2fffff'

In [202]:
# sample_shapes_with_stops[sample_shapes_with_stops['h3_9'] == manual_wesbrook_agronomy_index]

In [203]:
# sample_shapes_with_stops[sample_shapes_with_stops['stop_name'].str.lower().str.contains('wesbrook', na=False)]


In [204]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

def simulate_bus_positions(
    shapes_df: pd.DataFrame,
    start_time: datetime,
    end_time: datetime,
    interval_seconds: int = 60,
) -> pd.DataFrame:
    """
    Simulate bus movements for each shape_id between start_time and end_time.
    Returns a DataFrame with columns:
    [timestamp, bus_id, route_id, direction_id, latitude, longitude]
    """

    # Sort points
    shapes_df = shapes_df.sort_values(["shape_id", "shape_pt_sequence"])

    # We'll collect all simulated rows in a list of dicts
    all_rows = []

    def generate_positions_for_shape(shape_group: pd.DataFrame, route_id, direction_id):
        """
        Generate lat/lon positions from the shape_group in evenly spaced intervals,
        adding a random dwell time of 30-60s whenever we encounter a non-null stop_name.
        """

        bus_uuid = str(uuid.uuid4())

        # Prepare arrays for lat/lon
        coords = list(zip(shape_group["shape_pt_lat"].values, shape_group["shape_pt_lon"].values))

        total_time = (end_time - start_time).total_seconds()
        num_intervals = int(total_time // interval_seconds)

        # time_offset accumulates any dwell time from stops
        time_offset = 0.0

        for i in range(num_intervals):
            # Current 'base' timestamp (before dwell offsets)
            # i * interval_seconds + time_offset
            timestamp = start_time + timedelta(seconds=i * interval_seconds + time_offset)

            # fraction of route completed
            frac = i / max(num_intervals - 1, 1)

            # index in coords
            idx_float = frac * (len(coords) - 1)
            idx_lower = int(np.floor(idx_float))
            idx_upper = int(np.ceil(idx_float))

            if idx_lower == idx_upper:
                lat, lon = coords[idx_lower]
            else:
                # Linear interpolation
                lat_lower, lon_lower = coords[idx_lower]
                lat_upper, lon_upper = coords[idx_upper]
                ratio = idx_float - idx_lower
                lat = lat_lower + (lat_upper - lat_lower) * ratio
                lon = lon_lower + (lon_upper - lon_lower) * ratio

            # -------------
            # Check if this position is "close" to a stop row with a non-null stop_name
            # -------------
            # One simplistic approach: if idx_lower or idx_upper is a real stop, we add a dwell time.
            # In a real app, you might do a more precise check or only dwell when fraction is exactly a stop index, etc.
            stop_name_lower = shape_group.iloc[idx_lower]["stop_name"] if not pd.isnull(idx_lower) else None
            stop_name_upper = shape_group.iloc[idx_upper]["stop_name"] if not pd.isnull(idx_upper) else None

            # If either of those stops is non-null, add a random dwell
            if stop_name_lower is not None or stop_name_upper is not None:
                # Add a random dwell of 60-120s
                dwell = random.uniform(60, 120)
                time_offset += dwell
                # (Optionally) you could also hold the bus in the same lat/lon for that dwell time 
                # by repeating a row, but here's the simpler approach of just offsetting future times.

            # Add the row
            all_rows.append(
                {
                    "timestamp": timestamp,
                    # "bus_id": f"Bus{route_id}{direction_id}",  # or generate unique
                    "route_id": route_id,
                    "direction_id": direction_id,
                    "latitude": lat,
                    "longitude": lon,
                    "bus_id": bus_uuid,   
                    "elapsed_seconds":round((timestamp - start_time).total_seconds()),
                }
            )

    # -------------------------
    # MAIN LOOP: group by shape_id and generate forward/backward
    # -------------------------
    for shape_id, group in shapes_df.groupby("shape_id"):
        # Forward direction
        generate_positions_for_shape(group, shape_id, direction_id=1)
        
        # Backward direction: pass a reversed group
        generate_positions_for_shape(group.iloc[::-1], shape_id, direction_id=0)

    # Convert all_rows to DataFrame
    simulated_df = pd.DataFrame(all_rows)
    return simulated_df


In [205]:
def generate_multi_bus_dispatch():
    starting_time = 0 
    simulated_buses_multi = []

    for _ in range(20):
        random_dispatch = random.randint(10, 30)
        start = datetime(2025, 1, 12, 9, 0, 0) + timedelta(minutes=starting_time)
        end = datetime(2025, 1, 12, 10, 0, 0) + timedelta(minutes=starting_time)

        simulated_buses = simulate_bus_positions(sample_shapes_with_stops, start, end, interval_seconds=30)
        simulated_buses_multi.append(simulated_buses)

        starting_time += random_dispatch

    return pd.concat(simulated_buses_multi)


In [206]:
# simulated_buses_multi = generate_multi_bus_dispatch()

In [207]:
start = datetime(2025, 1, 12, 9, 0, 0)
end = datetime(2025, 1, 12, 10, 0, 0)

simulated_buses = simulate_bus_positions_og(sample_shapes_with_stops, start, end, interval_seconds=1)
simulated_buses

,timestamp,bus_id,route_id,direction_id,latitude,longitude
0,2025-01-12 09:00:00,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266003,-123.247825
1,2025-01-12 09:00:01,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266006,-123.247804
2,2025-01-12 09:00:02,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266009,-123.247784
3,2025-01-12 09:00:03,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266012,-123.247763
4,2025-01-12 09:00:04,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266015,-123.247743
...,...,...,...,...,...,...
316795,2025-01-12 09:59:55,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266991,-123.248256
316796,2025-01-12 09:59:56,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266974,-123.248303
316797,2025-01-12 09:59:57,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266958,-123.248350
316798,2025-01-12 09:59:58,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266941,-123.248397


In [208]:
simulated_buses.timestamp.max()

Timestamp('2025-01-12 09:59:59')

In [209]:
simulated_buses['manual_timediff'] = (simulated_buses['timestamp'] - datetime(2025, 1, 12, 9, 0, 0)).dt.total_seconds().astype(int)

In [210]:
simulated_buses['elapsed_seconds'] = simulated_buses['manual_timediff']

In [211]:
simulated_buses

,timestamp,bus_id,route_id,direction_id,latitude,longitude,manual_timediff,elapsed_seconds
0,2025-01-12 09:00:00,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266003,-123.247825,0,0
1,2025-01-12 09:00:01,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266006,-123.247804,1,1
2,2025-01-12 09:00:02,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266009,-123.247784,2,2
3,2025-01-12 09:00:03,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266012,-123.247763,3,3
4,2025-01-12 09:00:04,a4732d70-6181-481f-8c2f-9d31951eca8b,292028,1,49.266015,-123.247743,4,4
...,...,...,...,...,...,...,...,...
316795,2025-01-12 09:59:55,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266991,-123.248256,3595,3595
316796,2025-01-12 09:59:56,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266974,-123.248303,3596,3596
316797,2025-01-12 09:59:57,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266958,-123.248350,3597,3597
316798,2025-01-12 09:59:58,54fd42e0-a7a0-480e-b84e-8bfe68d6f87b,298165,0,49.266941,-123.248397,3598,3598


In [212]:
def print_df(df):
    df['manual_timediff'] = df['manual_timediff'].astype(int)
    df['elapsed_seconds'] = df['elapsed_seconds'].astype(int)
    
    grouped_times = [int(x) for x in df['manual_timediff'].unique()]
    grouped_times.sort()
    wait_times = [int(x) for x in np.diff(grouped_times)]
    wait_times.insert(0, 0)
    
    # Group data by elapsed_seconds
    for current_time, wait_time in zip(grouped_times, wait_times):
        # Get data for current timestamp
        current_data = df[df['manual_timediff'] == current_time]
        
        # Convert to list of dictionaries and ensure all values are JSON serializable
        data_to_send = current_data.to_dict(orient='records')

        print(data_to_send)

In [213]:
# print_df(simulated_buses)

In [214]:
# simulated_buses_multi.head(20)

In [215]:
simulated_buses.to_csv("../data/simulated/simulated_buses.csv", index=False)

In [216]:
def shapes_to_animated_geojson(shapes_df: pd.DataFrame) -> Dict:
    """
    Convert bus movement data into animated GeoJSON format for Kepler.gl.
    
    Args:
        shapes_df: DataFrame with columns timestamp, bus_id, route_id, direction_id,
                  latitude, longitude
    
    Returns:
        Dictionary in GeoJSON format with timestamps for animation
    """
    # Sort by bus_id and timestamp to ensure correct sequence
    shapes_df = shapes_df.sort_values(["bus_id", "timestamp"])
    
    # Convert timestamps to Unix time (seconds since epoch)
    shapes_df['unix_time'] = shapes_df['timestamp'].astype('int64') // 10**9
    
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }
    
    # Group points by bus_id
    for bus_id, points in shapes_df.groupby("bus_id"):
        # Get the route_id and direction_id
        route_id = points["route_id"].iloc[0]
        direction_id = points["direction_id"].iloc[0]
        
        # Create coordinates list with [longitude, latitude, altitude, timestamp]
        coordinates = points.apply(
            lambda row: [
                float(row["longitude"]), 
                float(row["latitude"]), 
                0,  # altitude (set to 0)
                int(row["unix_time"])  # timestamp in seconds
            ], 
            axis=1
        ).tolist()
        
        # Create feature with properties
        feature = {
            "type": "Feature",
            "properties": {
                "bus_id": bus_id,
                "route_id": route_id,
                "direction_id": direction_id
            },
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            }
        }
        geojson["features"].append(feature)
    
    return geojson

# Example usage with custom encoder for proper serialization
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        if isinstance(obj, (np.int64, np.int32)):
            return int(obj)
        if isinstance(obj, (np.float64, np.float32)):
            return float(obj)
        return super().default(obj)

# Convert and save
def save_animated_geojson(df: pd.DataFrame, output_file: str):
    """
    Convert DataFrame to animated GeoJSON and save to file.
    
    Args:
        df: Input DataFrame with bus movement data
        output_file: Path to save the GeoJSON file
    """
    geojson_data = shapes_to_animated_geojson(df)
    
    with open(output_file, 'w') as f:
        json.dump(geojson_data, f, cls=CustomEncoder, indent=2)

# Usage example:
# save_animated_geojson(df, 'animated_bus_routes.geojson')

In [217]:
save_animated_geojson(simulated_buses, '../data/simulated/simulated_buses.geojson')

In [218]:
import pandas as pd
import folium
import random
from typing import Optional


def create_bus_map(
    df: pd.DataFrame, zoom_level: int = 18, save_path: Optional[str] = None
) -> folium.Map:
    """
    Create an interactive map showing bus routes with distinct colors for each bus.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing columns: 'timestamp', 'bus_id', 'route_id',
        'latitude', 'longitude'
    zoom_level : int, optional (default=18)
        Initial zoom level for the map
    save_path : str, optional (default=None)
        If provided, saves the map to this path (e.g., 'bus_map.html')

    Returns:
    --------
    folium.Map
        The created map object
    """

    # Validate required columns
    required_columns = ["timestamp", "bus_id", "route_id", "latitude", "longitude"]
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Create a map centered on the mean coordinates
    center_lat = df["latitude"].mean()
    center_lon = df["longitude"].mean()
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_level,
        tiles="CartoDB positron",
        control_scale=True,
    )

    # Generate a color for each unique bus_id
    unique_buses = df["bus_id"].unique()
    colors = {
        bus: "#{:06x}".format(random.randint(0, 0xFFFFFF)) for bus in unique_buses
    }

    # Add markers and path for each bus
    for bus_id in unique_buses:
        bus_data = df[df["bus_id"] == bus_id]

        # Create a path for the bus route
        points = [[row["latitude"], row["longitude"]] for _, row in bus_data.iterrows()]

        # Add the polyline for the bus route
        folium.PolyLine(points, weight=3, color=colors[bus_id], opacity=0.8).add_to(m)

        # Add markers for each position
        for _, row in bus_data.iterrows():
            folium.CircleMarker(
                location=[row["latitude"], row["longitude"]],
                radius=8,
                color=colors[bus_id],
                fill=True,
                popup=f"""
                    <b>Bus Information</b><br>
                    ID: {row['bus_id']}<br>
                    Time: {row['timestamp']}<br>
                    Route: {row['route_id']}
                """,
                tooltip=f"Bus {row['bus_id']}",
            ).add_to(m)

    # Add a legend
    legend_html = """
        <div style="position: fixed; 
                    bottom: 50px; right: 50px; 
                    width: 150px; 
                    border:2px solid grey; 
                    z-index:9999; 
                    background-color:white;
                    opacity:0.8;
                    font-size:12px;
                    padding:10px">
            <p><b>Bus Routes</b></p>
    """

    for bus_id, color in colors.items():
        legend_html += f'<p><span style="color:{color};">●</span> {bus_id}</p>'

    legend_html += "</div>"
    m.get_root().html.add_child(folium.Element(legend_html))

    # Save the map if path is provided
    if save_path:
        m.save(save_path)

    return m


# bus_map = create_bus_map(
#     simulated_buses,
# )

In [225]:
manual_socket = [
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "2d4813d2-01de-4008-a74c-6ab6875e1a18",
            "route_id": 292028,
            "direction_id": 1,
            "latitude": 49.26600883495415,
            "longitude": -123.24778398860796,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "1fc5eb39-dd08-4810-a85a-28f93ab417f0",
            "route_id": 292028,
            "direction_id": 0,
            "latitude": 49.28164393970548,
            "longitude": -123.11002798944152,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "74fc7a98-f35d-40b5-89cb-ca5eda6fb082",
            "route_id": 292029,
            "direction_id": 1,
            "latitude": 49.26601132453459,
            "longitude": -123.247766490414,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "ee08e8fa-acff-454b-91f1-c58185af96a0",
            "route_id": 292029,
            "direction_id": 0,
            "latitude": 49.2877079297027,
            "longitude": -123.04637854876354,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "4d11f643-6c71-43e1-8747-6a16d23935f0",
            "route_id": 292030,
            "direction_id": 1,
            "latitude": 49.26600883495415,
            "longitude": -123.24778398860796,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "4aa143d3-1109-4772-80c7-b71a104772c6",
            "route_id": 292030,
            "direction_id": 0,
            "latitude": 49.28164393970548,
            "longitude": -123.11002798944152,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "4969a0af-5127-4199-a24b-4081e3af589c",
            "route_id": 292031,
            "direction_id": 1,
            "latitude": 49.26600891275354,
            "longitude": -123.2477834417894,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "c4cb551b-a728-4caf-91de-cc63852ab471",
            "route_id": 292031,
            "direction_id": 0,
            "latitude": 49.28164504556822,
            "longitude": -123.11002748263408,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "c8d855d2-3c60-4914-bca0-b65d36518e13",
            "route_id": 292084,
            "direction_id": 1,
            "latitude": 49.26570092553487,
            "longitude": -123.24972617004724,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "0701236e-97b8-4044-8be6-1249e0d685ab",
            "route_id": 292084,
            "direction_id": 0,
            "latitude": 49.2653465712698,
            "longitude": -123.02493264184496,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "965901da-b05f-4570-bf84-9986521691ff",
            "route_id": 292130,
            "direction_id": 1,
            "latitude": 49.26590819199778,
            "longitude": -123.24847076771326,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "2e59b305-9336-4541-8204-1a7689e7eec2",
            "route_id": 292130,
            "direction_id": 0,
            "latitude": 49.2812999747152,
            "longitude": -123.02653643011948,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "999eb1c7-290c-4d7e-9539-738a1d188921",
            "route_id": 292132,
            "direction_id": 1,
            "latitude": 49.2659050111142,
            "longitude": -123.2484924337316,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "e7905bab-b682-4324-bc25-b8db57ab9699",
            "route_id": 292132,
            "direction_id": 0,
            "latitude": 49.28336395081968,
            "longitude": -123.11101236732426,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "ccb5c31d-2d10-40be-8bb6-cec37c733a38",
            "route_id": 292133,
            "direction_id": 1,
            "latitude": 49.26590495220895,
            "longitude": -123.24849283495416,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "d5ff2dd9-d1c7-4304-a8de-9dada47bd889",
            "route_id": 292133,
            "direction_id": 0,
            "latitude": 49.28336431147541,
            "longitude": -123.1110118205057,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "606c9334-aca4-460f-903d-c6314e62db27",
            "route_id": 292249,
            "direction_id": 1,
            "latitude": 49.26737471380939,
            "longitude": -123.24718016310086,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "845a21eb-bb7c-439e-97ad-8370881d7a83",
            "route_id": 292249,
            "direction_id": 0,
            "latitude": 49.24799336537927,
            "longitude": -123.05665310558489,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "e0848aaf-ebee-40a0-b84e-ba883d6781e6",
            "route_id": 292250,
            "direction_id": 1,
            "latitude": 49.26737885495971,
            "longitude": -123.24716616421227,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "cc0d36d0-52cf-4d3a-96c3-34065871e88a",
            "route_id": 292250,
            "direction_id": 0,
            "latitude": 49.266379047791055,
            "longitude": -123.00370337788274,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "27b22dee-a8d1-49d1-93f5-5bd50d327d05",
            "route_id": 292277,
            "direction_id": 1,
            "latitude": 49.26748828535705,
            "longitude": -123.24765532953596,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "0611763b-5146-4935-a88b-592e098564f9",
            "route_id": 292277,
            "direction_id": 0,
            "latitude": 49.244308141983886,
            "longitude": -123.04543253820506,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "0c6c4813-06e5-464a-8460-a531ff2a869b",
            "route_id": 292288,
            "direction_id": 1,
            "latitude": 49.26727208613504,
            "longitude": -123.248238252848,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "17e170af-ac7b-4129-9c6f-3564326a635a",
            "route_id": 292288,
            "direction_id": 0,
            "latitude": 49.2852742072798,
            "longitude": -123.11193059016394,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "6be21dcd-97b2-44d9-9e10-a7ed17041978",
            "route_id": 292289,
            "direction_id": 1,
            "latitude": 49.26726585607113,
            "longitude": -123.24825577104752,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "1faea264-26af-4dfb-85c0-e3e3eebf80da",
            "route_id": 292289,
            "direction_id": 0,
            "latitude": 49.334276129758265,
            "longitude": -123.17970842233952,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "586568e2-4851-4cec-a143-9667c4d49290",
            "route_id": 292297,
            "direction_id": 1,
            "latitude": 49.26722945901639,
            "longitude": -123.24758921311476,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "e7ae3cb5-9188-4927-b720-bc7226799c63",
            "route_id": 292297,
            "direction_id": 0,
            "latitude": 49.22608421311476,
            "longitude": -123.004005,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "07d262db-0ebd-45f3-b5ac-ee8efef2a02b",
            "route_id": 292316,
            "direction_id": 1,
            "latitude": 49.26749574298417,
            "longitude": -123.24763519394276,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "b78ebf8f-c9ac-46ca-829f-bd715ba61858",
            "route_id": 292316,
            "direction_id": 0,
            "latitude": 49.25544120366769,
            "longitude": -123.23600936148932,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "7fccd02e-2567-48fa-aabc-60acb43e6b58",
            "route_id": 292319,
            "direction_id": 1,
            "latitude": 49.267489385662685,
            "longitude": -123.24765235871077,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "6a6cd0f6-c6e5-4677-af9f-bbdf9e6ac305",
            "route_id": 292319,
            "direction_id": 0,
            "latitude": 49.26595361350376,
            "longitude": -123.07951008502364,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "ad6d72b4-6649-4cd7-9fd7-21e1315cad6a",
            "route_id": 292321,
            "direction_id": 1,
            "latitude": 49.26746267101973,
            "longitude": -123.24688079911088,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "e1d969db-8f0b-4593-baee-99efeaaa3c0e",
            "route_id": 292321,
            "direction_id": 0,
            "latitude": 49.26654123673242,
            "longitude": -123.02345991942208,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "66ee2694-ab3a-41a3-a855-6245c645d3b4",
            "route_id": 292322,
            "direction_id": 1,
            "latitude": 49.267461226173936,
            "longitude": -123.24688620061129,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "d47e77a5-b117-4ef7-b87e-0bda91220c56",
            "route_id": 292322,
            "direction_id": 0,
            "latitude": 49.2626487057516,
            "longitude": -123.06707057710476,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "842b6439-f746-4377-a62f-4032386cab53",
            "route_id": 292323,
            "direction_id": 1,
            "latitude": 49.26746267101973,
            "longitude": -123.24688079911088,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "b0745add-6726-4e29-931a-f600ec5c1bda",
            "route_id": 292323,
            "direction_id": 0,
            "latitude": 49.26654123673242,
            "longitude": -123.02345991942208,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "a7a55020-81b8-445d-baba-fae8692eb725",
            "route_id": 292325,
            "direction_id": 1,
            "latitude": 49.267461226173936,
            "longitude": -123.24688620061129,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "7b2d361a-e1ee-43a0-818c-c2d2481abf18",
            "route_id": 292325,
            "direction_id": 0,
            "latitude": 49.2626487057516,
            "longitude": -123.06707057710476,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "b57ee26e-b68e-4475-bef9-6c79c7d7afe7",
            "route_id": 293191,
            "direction_id": 1,
            "latitude": 49.267273208669074,
            "longitude": -123.24823509641568,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "8b74e8a1-f06a-4506-8ffb-f19fab7e4150",
            "route_id": 293191,
            "direction_id": 0,
            "latitude": 49.28175083328702,
            "longitude": -123.11724955265352,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "4c46b982-aaf6-4d9b-b8e6-fd4f8f6887ad",
            "route_id": 293231,
            "direction_id": 1,
            "latitude": 49.26695757599333,
            "longitude": -123.24834998721867,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "1006007f-0116-49b9-889b-5855f3e51a00",
            "route_id": 293231,
            "direction_id": 0,
            "latitude": 49.23869697415949,
            "longitude": -123.03287045762713,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "6c340838-5ab4-42f8-9e4d-5df696ce30f2",
            "route_id": 296963,
            "direction_id": 1,
            "latitude": 49.26600891275354,
            "longitude": -123.2477834417894,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "1926f915-d541-4780-9f4d-2eb659e47c66",
            "route_id": 296963,
            "direction_id": 0,
            "latitude": 49.28164504556822,
            "longitude": -123.11002748263408,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "3e11d991-9694-41d2-b36f-cad20c6aa2ef",
            "route_id": 296964,
            "direction_id": 1,
            "latitude": 49.26600883495415,
            "longitude": -123.24778398860796,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "fab94d96-49a7-4e4a-bfa2-4759ed611430",
            "route_id": 296964,
            "direction_id": 0,
            "latitude": 49.28164393970548,
            "longitude": -123.11002798944152,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "49a0e123-80a4-4326-b468-19d23c5c1956",
            "route_id": 296965,
            "direction_id": 1,
            "latitude": 49.26601132453459,
            "longitude": -123.247766490414,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "695dc228-7540-4c3a-9f5f-80b970307376",
            "route_id": 296965,
            "direction_id": 0,
            "latitude": 49.2877079297027,
            "longitude": -123.04637854876354,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "ecf5c4bf-63d6-46ff-b2a3-1a3eb5e63b82",
            "route_id": 296966,
            "direction_id": 1,
            "latitude": 49.26600883495415,
            "longitude": -123.24778398860796,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "99e3cd7a-9561-4f54-8b05-bf9d64eb5d5e",
            "route_id": 296966,
            "direction_id": 0,
            "latitude": 49.28164393970548,
            "longitude": -123.11002798944152,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "a4737901-c5ec-4bbb-afda-871e9fa13df8",
            "route_id": 297021,
            "direction_id": 1,
            "latitude": 49.26570092553487,
            "longitude": -123.24972617004724,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "e1f28e80-1db5-487c-a62f-fb00ec358bb3",
            "route_id": 297021,
            "direction_id": 0,
            "latitude": 49.2653465712698,
            "longitude": -123.02493264184496,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "45c64642-ef1b-4c66-b627-1a237db31144",
            "route_id": 297057,
            "direction_id": 1,
            "latitude": 49.26590495220895,
            "longitude": -123.24849283495416,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "67d0594a-976d-4e91-afb2-db4f9492b529",
            "route_id": 297057,
            "direction_id": 0,
            "latitude": 49.28336431147541,
            "longitude": -123.1110118205057,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "0cd244df-f62d-42c8-8886-be9ea6c0c5fa",
            "route_id": 297058,
            "direction_id": 1,
            "latitude": 49.2659050111142,
            "longitude": -123.2484924337316,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "51cfcdce-d39f-4891-9643-de05427f1f4a",
            "route_id": 297058,
            "direction_id": 0,
            "latitude": 49.28336395081968,
            "longitude": -123.11101236732426,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "818ba86e-5bd0-47c0-b490-757742a61ab5",
            "route_id": 297060,
            "direction_id": 1,
            "latitude": 49.26590819199778,
            "longitude": -123.24847076771326,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "69ca11d0-473c-470c-9d19-094ed2dec7bf",
            "route_id": 297060,
            "direction_id": 0,
            "latitude": 49.2812999747152,
            "longitude": -123.02653643011948,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "569fbaab-f3d2-473e-884c-ee5b522ee37d",
            "route_id": 297179,
            "direction_id": 1,
            "latitude": 49.26737885495971,
            "longitude": -123.24716616421227,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "07303ea8-62d0-442b-916a-a9ca3bf1671e",
            "route_id": 297179,
            "direction_id": 0,
            "latitude": 49.266379047791055,
            "longitude": -123.00370337788274,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "da69c02f-2770-466e-a591-729dc6cc7db7",
            "route_id": 297180,
            "direction_id": 1,
            "latitude": 49.26737471380939,
            "longitude": -123.24718016310086,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "45a18f8e-c581-4766-8bfa-ac6e2166a820",
            "route_id": 297180,
            "direction_id": 0,
            "latitude": 49.24799336537927,
            "longitude": -123.05665310558489,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "593d5193-495c-4ef5-b975-ced7a9bbdd99",
            "route_id": 297211,
            "direction_id": 1,
            "latitude": 49.26748828535705,
            "longitude": -123.24765532953596,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "283da8db-bc3e-41ae-9eae-5cebbe99e2f1",
            "route_id": 297211,
            "direction_id": 0,
            "latitude": 49.244308141983886,
            "longitude": -123.04543253820506,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "3e118164-78a2-47b7-8acf-0a642793cd5b",
            "route_id": 297222,
            "direction_id": 1,
            "latitude": 49.26726585607113,
            "longitude": -123.24825577104752,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "050c9882-fd8d-4e55-9dc3-1e21caf68661",
            "route_id": 297222,
            "direction_id": 0,
            "latitude": 49.334276129758265,
            "longitude": -123.17970842233952,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "a6dd0b37-8e26-46db-b44c-756f8bd1d02a",
            "route_id": 297223,
            "direction_id": 1,
            "latitude": 49.26727208613504,
            "longitude": -123.248238252848,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "d8007fad-2aa7-47b7-9fd0-404a7680b428",
            "route_id": 297223,
            "direction_id": 0,
            "latitude": 49.2852742072798,
            "longitude": -123.11193059016394,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "adff1d05-68dc-4a52-ad16-7faaaf4c2690",
            "route_id": 297226,
            "direction_id": 1,
            "latitude": 49.26722945901639,
            "longitude": -123.24758921311476,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "7628eb42-692b-44fd-8cf7-7da05a6f16c4",
            "route_id": 297226,
            "direction_id": 0,
            "latitude": 49.22608421311476,
            "longitude": -123.004005,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "15a34064-80f5-4b61-ae0e-542cb8c0bc88",
            "route_id": 297250,
            "direction_id": 1,
            "latitude": 49.26749574298417,
            "longitude": -123.24763519394276,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "d885b22e-d1bd-4e4d-b4b7-4bed5fc28f7e",
            "route_id": 297250,
            "direction_id": 0,
            "latitude": 49.25544120366769,
            "longitude": -123.23600936148932,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "eb03ec8a-18ec-4fd2-86d2-1f88e49bf049",
            "route_id": 297253,
            "direction_id": 1,
            "latitude": 49.267489385662685,
            "longitude": -123.24765235871077,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "8800a278-c2bb-46c9-8833-14d61e18c9f6",
            "route_id": 297253,
            "direction_id": 0,
            "latitude": 49.26595361350376,
            "longitude": -123.07951008502364,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "fe7afcee-8d08-4dab-801d-d1e6885edf10",
            "route_id": 297255,
            "direction_id": 1,
            "latitude": 49.267461226173936,
            "longitude": -123.24688620061129,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "2b5f5811-b5cf-4b3c-a4a1-b7f82c0f3ec2",
            "route_id": 297255,
            "direction_id": 0,
            "latitude": 49.2626487057516,
            "longitude": -123.06707057710476,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "0ad89fea-29ec-457a-983d-b6a9c3bb02d2",
            "route_id": 297257,
            "direction_id": 1,
            "latitude": 49.26746267101973,
            "longitude": -123.24688079911088,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "a34e2ac1-dc79-4d20-a4f2-f91d9875d7a5",
            "route_id": 297257,
            "direction_id": 0,
            "latitude": 49.26654123673242,
            "longitude": -123.02345991942208,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "e6d34000-e155-493f-9951-f3e53b62614a",
            "route_id": 297258,
            "direction_id": 1,
            "latitude": 49.267461226173936,
            "longitude": -123.24688620061129,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "aba9f1af-6a41-455a-aef3-8106c6275453",
            "route_id": 297258,
            "direction_id": 0,
            "latitude": 49.2626487057516,
            "longitude": -123.06707057710476,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "c69aff4e-ffeb-4709-8081-23e26d450ed7",
            "route_id": 297259,
            "direction_id": 1,
            "latitude": 49.26746267101973,
            "longitude": -123.24688079911088,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "435ac1a0-2af8-4ffe-beea-b021c70d41e1",
            "route_id": 297259,
            "direction_id": 0,
            "latitude": 49.26654123673242,
            "longitude": -123.02345991942208,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "298269f8-b6b8-4caa-9395-c67159d37a8a",
            "route_id": 298129,
            "direction_id": 1,
            "latitude": 49.267273208669074,
            "longitude": -123.24823509641568,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "7fa55bcb-79ff-4563-bd5e-5cb01f5d8868",
            "route_id": 298129,
            "direction_id": 0,
            "latitude": 49.28175083328702,
            "longitude": -123.11724955265352,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "86922eb4-5ac5-4704-aaf1-6345b7c9bf9a",
            "route_id": 298165,
            "direction_id": 1,
            "latitude": 49.26695757599333,
            "longitude": -123.24834998721867,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        },
        {
            "timestamp": "2025-01-12 09:00:02",
            "bus_id": "7a01ee0b-ddca-4ec5-98ca-8a1a0d9e8d27",
            "route_id": 298165,
            "direction_id": 0,
            "latitude": 49.23869697415949,
            "longitude": -123.03287045762713,
            "manual_timediff": 2,
            "elapsed_seconds": 2
        }
    ]

In [227]:
pd.DataFrame(manual_socket)

,timestamp,bus_id,route_id,direction_id,latitude,longitude,manual_timediff,elapsed_seconds
0,2025-01-12 09:00:02,2d4813d2-01de-4008-a74c-6ab6875e1a18,292028,1,49.266009,-123.247784,2,2
1,2025-01-12 09:00:02,1fc5eb39-dd08-4810-a85a-28f93ab417f0,292028,0,49.281644,-123.110028,2,2
2,2025-01-12 09:00:02,74fc7a98-f35d-40b5-89cb-ca5eda6fb082,292029,1,49.266011,-123.247766,2,2
3,2025-01-12 09:00:02,ee08e8fa-acff-454b-91f1-c58185af96a0,292029,0,49.287708,-123.046379,2,2
4,2025-01-12 09:00:02,4d11f643-6c71-43e1-8747-6a16d23935f0,292030,1,49.266009,-123.247784,2,2
...,...,...,...,...,...,...,...,...
83,2025-01-12 09:00:02,435ac1a0-2af8-4ffe-beea-b021c70d41e1,297259,0,49.266541,-123.023460,2,2
84,2025-01-12 09:00:02,298269f8-b6b8-4caa-9395-c67159d37a8a,298129,1,49.267273,-123.248235,2,2
85,2025-01-12 09:00:02,7fa55bcb-79ff-4563-bd5e-5cb01f5d8868,298129,0,49.281751,-123.117250,2,2
86,2025-01-12 09:00:02,86922eb4-5ac5-4704-aaf1-6345b7c9bf9a,298165,1,49.266958,-123.248350,2,2


In [228]:
create_bus_map(pd.DataFrame(manual_socket))